In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

base_url = "https://arxiv.org/search/?query=post-quantum+cryptography&searchtype=all&abstracts=show&order=-announced_date_first&size=50&start="

num_pages = 5  # 50 x 5 = 250 papers
results = []

for page in range(num_pages):
    url = base_url + str(page * 50)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    entries = soup.find_all("li", class_="arxiv-result")
    for entry in entries:
        title = entry.find("p", class_="title").text.strip()
        authors = [a.text for a in entry.find("p", class_="authors").find_all("a")]
        date = entry.find("p", class_="is-size-7").text.split(";")[0].replace("Submitted ", "").strip()
        results.append({"title": title, "authors": authors, "date": date})

    time.sleep(1)

# Save full paper dataset
df_papers = pd.DataFrame(results)
#df_papers.to_csv("post_quantum_papers_detailed.csv", index=False)

# Author Count
flat_authors = [author for paper in results for author in paper["authors"]]
author_counts = pd.Series(flat_authors).value_counts().reset_index()
author_counts.columns = ["Author", "Paper Count"]
author_counts.to_csv("top_post_quantum_authors.csv", index=False)
